In [216]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

In [223]:
webpage = 'https://www.snezak.si/razmere/?date=season&startDate=&endDate=&conditionsMin=1&conditionsMax=5&safetyMin=1&safetyMax=5'
r = requests.get(webpage)
c = r.content
soup = BeautifulSoup(c, 'lxml')

In [218]:
# find classes in soup
get_routes = soup.find_all('div', class_ = "route")
get_authors = soup.find_all('div', class_ = "author")
get_dates = soup.find_all('time')
get_ratings = soup.find_all('div', class_= 'rate')

In [255]:
# get data
routes = [x.text.translate({ord('\t'): None, ord('\n'): None}) for x in get_routes]
authors = [x.text.translate({ord('\t'): None, ord('\n'): None}) for x in get_authors]
dates = [i['datetime'] for i in get_dates if i.has_attr('datetime')]
condition_ratings = [len(x.find_all('img')) for x in get_ratings if x.find(class_= 'title').text == 'Smučarske razmere']
safety_ratings = [len(x.find_all('img')) for x in get_ratings if x.find(class_= 'title').text == 'Občutek varnosti']

In [256]:
df = pd.DataFrame({'author': authors, 'date': dates, 
                   'safety': safety_ratings, 'conditions': condition_ratings})

df['date'] = pd.to_datetime(df['date'])

df[['region','mountain']] = pd.Series(routes).str.split('|',expand=True).loc[:,:1].apply(lambda x: x.str.strip())

df = df[['date','author','region','mountain','conditions','safety']]

In [257]:
df.head()

,date,author,region,mountain,conditions,safety
0,2018-05-19,Marijana & Marko,Visoke Ture,Fuscherkarkopf,3,2
1,2018-05-19,Barbara Dekleva Jenčič,Visoke Ture,Kölnbreinspitze,5,4
2,2018-05-17,Primoz Vrhovnik,Julijske Alpe,Jugova grapa,4,5
3,2018-05-13,Marijana & Marko,Visoke Ture,Racherin,3,4
4,2018-05-13,Barbara Dekleva Jenčič,Visoke Ture,Oberlercher Spitze,4,5


In [243]:
# tested 20/80 rule for reports from different authors. Top 20% of authors post 66% of articles.